<a href="https://colab.research.google.com/github/samvardhan777/financial_banking_app_sentimental_analysis/blob/main/fine_tune_BERT_banking_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!mkdir ~/.kaggle

In [4]:
!cp /content/drive/MyDrive/kaggle_api_credentials/kaggle.json ~/.kaggle/

In [5]:
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download vgs777/financial-label-sentiment-analysis-banking-app

  0% 0.00/18.2k [00:00<?, ?B/s]
100% 18.2k/18.2k [00:00<00:00, 37.0MB/s]


In [7]:
!unzip /content/financial-label-sentiment-analysis-banking-app.zip

Archive:  /content/financial-label-sentiment-analysis-banking-app.zip
  inflating: final_bfsi_lablel.csv   


In [8]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [9]:
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.8 MB/s eta 0:00:00


# Load Model

In [10]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset

import pandas as pd
# Dataloaders for pytorch models
from torch.utils.data import DataLoader, TensorDataset
# BERT models tokenizer and classifier (Auto detect the underlying models (Bert-base, Camembert, bert-tiny...)
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW

# LoraConfig, PeftConfig and SFTTrainer are used for precise FineTuning
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
# AutoModelForCausalLM and AutoTokenizer are for LLMs, BitsAndBytesConfig is for quantization and pipeline is for simplifying LLMs code
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline,
                          logging)
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)
from sklearn.model_selection import train_test_split

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
modelname = "prajjwal1/bert-tiny"
tokenizer = AutoTokenizer.from_pretrained(modelname)
model = AutoModelForSequenceClassification.from_pretrained(modelname, num_labels=3)
model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-12, e

In [12]:
MAX_SEQ_LENGTH = 128
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
EPOCHS = 10

# Dataset Creation

In [13]:
import pandas as pd
df = pd.read_csv('final_bfsi_lablel.csv')

In [14]:
df

,Financial_context,annotation_id,annotator,content,created_at,id,language,lead_time,score,sentiment,thumbsUpCount,updated_at
0,Financial,14,1,1] Customer service is really nice.,2024-03-31T06:58:47.318794Z,1332,en,9.348,3,Positive,0,2024-03-31T06:58:47.318813Z
1,Non Financial,15,1,2] It is not very easy to set up mobile bankin...,2024-03-31T06:59:03.859506Z,1333,en,12.568,3,Neutral,0,2024-03-31T06:59:03.859527Z
2,Non Financial,16,1,I understand it's for safety but still a hassle.,2024-03-31T06:59:46.583294Z,1334,en,21.842,3,Neutral,0,2024-03-31T06:59:46.583307Z
3,Non Financial,17,1,3] App itself is easy to use.,2024-03-31T06:59:58.593876Z,1335,en,7.949,3,Positive,0,2024-03-31T06:59:58.593889Z
4,Non Financial,18,1,"4] Unfortunately, you can't view your card num...",2024-03-31T07:00:26.848093Z,1336,en,16.955,3,Negative,0,2024-03-31T07:00:26.848109Z
...,...,...,...,...,...,...,...,...,...,...,...,...
383,Non Financial,382,1,I don't know what's is the problem?,2024-03-31T11:39:35.667314Z,2532,en,9.276,4,Negative,0,2024-03-31T11:39:35.667327Z
384,Financial,383,1,It's very disappointing.,2024-03-31T11:39:43.659762Z,2533,en,6.336,4,Negative,0,2024-03-31T11:39:43.659780Z
385,Non Financial,370,1,2nd largest bank in India but mobile app is no...,2024-03-31T11:29:04.279001Z,2534,en,4.080,4,Negative,0,2024-03-31T11:29:04.279018Z
386,Financial,384,1,What a pity.,2024-03-31T11:40:03.224609Z,2535,en,25.356,4,Negative,0,2024-03-31T11:40:03.224623Z


In [15]:
final_df = df[['Financial_context','sentiment','content']]

In [16]:
final_df

,Financial_context,sentiment,content
0,Financial,Positive,1] Customer service is really nice.
1,Non Financial,Neutral,2] It is not very easy to set up mobile bankin...
2,Non Financial,Neutral,I understand it's for safety but still a hassle.
3,Non Financial,Positive,3] App itself is easy to use.
4,Non Financial,Negative,"4] Unfortunately, you can't view your card num..."
...,...,...,...
383,Non Financial,Negative,I don't know what's is the problem?
384,Financial,Negative,It's very disappointing.
385,Non Financial,Negative,2nd largest bank in India but mobile app is no...
386,Financial,Negative,What a pity.


In [17]:
texts   = list(df["content"])
labels  = list(df["sentiment"])
ulabels = list(set(labels))
# we need to encode the labels [for example --> 0:positive, 1:negative, 2:neutral  ]
dic = {ulabels[i]:i for i in range(len(ulabels))}
labels = [dic[labels[i]] for i in range(len(labels))]

In [19]:
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
train_inputs, test_inputs, train_labels, test_labels = train_test_split(inputs["input_ids"], labels, test_size=0.2, random_state=42)
train_inputs, val_inputs, train_labels, val_labels = train_test_split(train_inputs, train_labels, test_size=0.2, random_state=42)

In [36]:
from torch.utils.data import Dataset, DataLoader
import torch

class SentimentDataset(Dataset):
    def __init__(self, data, labels):
        self.samples = data
        self.labels = labels

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.item()  # Convert tensor to integer if necessary

        sample = self.samples[idx]
        label = self.labels[idx]
        return sample, label

# Assume train_inputs, train_labels, etc. are already tensors
train_dataset = SentimentDataset(train_inputs, train_labels)
val_dataset = SentimentDataset(val_inputs, val_labels)
test_dataset = SentimentDataset(test_inputs, test_labels)

# DataLoader
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Assuming 'model' is defined, 'device' is set, and 'optimizer' is configured
def train(num_epochs=50):
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        for batch in train_loader:
            input_ids, labels = [b.to(device) for b in batch]
            optimizer.zero_grad()
            outputs = model(input_ids=input_ids, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()
            loss.backward()
            optimizer.step()

        avg_train_loss = total_loss / len(train_loader)

        # Validation loop
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for batch in val_loader:
                input_ids, labels = [b.to(device) for b in batch]
                outputs = model(input_ids=input_ids, labels=labels)
                val_loss += outputs.loss.item()
                _, predicted = torch.max(outputs.logits, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = correct / total

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.2%}")

In [37]:
train()

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Epoch 1/50, Train Loss: 1.0654, Val Loss: 1.0490, Val Acc: 40.32%
Epoch 2/50, Train Loss: 1.0350, Val Loss: 1.0259, Val Acc: 45.16%
Epoch 3/50, Train Loss: 1.0046, Val Loss: 1.0042, Val Acc: 50.00%
Epoch 4/50, Train Loss: 0.9865, Val Loss: 0.9854, Val Acc: 61.29%
Epoch 5/50, Train Loss: 0.9697, Val Loss: 0.9688, Val Acc: 58.06%
Epoch 6/50, Train Loss: 0.9438, Val Loss: 0.9523, Val Acc: 59.68%
Epoch 7/50, Train Loss: 0.9296, Val Loss: 0.9345, Val Acc: 62.90%
Epoch 8/50, Train Loss: 0.9148, Val Loss: 0.9139, Val Acc: 64.52%
Epoch 9/50, Train Loss: 0.8967, Val Loss: 0.8949, Val Acc: 64.52%
Epoch 10/50, Train Loss: 0.8702, Val Loss: 0.8790, Val Acc: 69.35%
Epoch 11/50, Train Loss: 0.8602, Val Loss: 0.8604, Val Acc: 69.35%
Epoch 12/50, Train Loss: 0.8415, Val Loss: 0.8455, Val Acc: 69.35%
Epoch 13/50, Train Loss: 0.8322, Val Loss: 0.8339, Val Acc: 70.97%
Epoch 14/50, Train Loss: 0.8237, Val Loss: 0.8215, Val Acc: 70.97%
Epoch 15/50, Train Loss: 0.8014, Val Loss: 0.8088, Val Acc: 69.35%
Epoc

In [38]:
import torch
from sklearn.metrics import classification_report

def test():
    model.eval()  # Set the model to evaluation mode
    test_loss = 0
    correct = 0
    total = 0
    true, pred = [], []

    with torch.no_grad():  # No need to track gradients during evaluation
        for batch in test_loader:
            input_ids, labels = batch
            # Move data to the appropriate device
            input_ids = input_ids.to(device)
            labels = labels.to(device)

            # Collect true labels for classification report
            true.extend(labels.cpu().numpy())  # Ensure data is moved to CPU and converted to NumPy

            outputs = model(input_ids=input_ids, labels=labels)
            test_loss += outputs.loss.item()

            _, predicted = torch.max(outputs.logits, 1)
            # Collect predictions for classification report
            pred.extend(predicted.cpu().numpy())  # Ensure data is moved to CPU and converted to NumPy

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_test_loss = test_loss / len(test_loader)
    test_accuracy = correct / total

    print(f"Test Loss: {avg_test_loss:.4f}, Test Acc: {test_accuracy:.2%}")

    # Compute the classification report
    class_report = classification_report(true, pred, digits=4)
    print('\nClassification Report:')
    print(class_report)

In [40]:
test()

Test Loss: 0.5004, Test Acc: 80.77%

Classification Report:
              precision    recall  f1-score   support

           0     0.7647    0.8387    0.8000        31
           1     0.0000    0.0000    0.0000         6
           2     0.8409    0.9024    0.8706        41

    accuracy                         0.8077        78
   macro avg     0.5352    0.5804    0.5569        78
weighted avg     0.7459    0.8077    0.7756        78



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
